In [ ]:
# import the required libs
import numpy as np
import pylab as pl
import commonUtil
import pandas as pd
import os
import datetime
import statsmodels.api as sm
from scipy.interpolate import spline
import warnings

warnings.filterwarnings("ignore")

#To get all query conditions into a dictionary
queryCondition = commonUtil.handle_query_condition()
ISOK_ALL_PARA = queryCondition['ISOK_ALL_PARA']
REPORT_TYPE = queryCondition['REPORT_TYPE']

DB_SET = queryCondition['DB_SET']
START_TIME = queryCondition['START_TIME']
END_TIME = queryCondition['END_TIME']
INTERVAL = queryCondition['INTERVAL']
TBSP_NAME = queryCondition['TBSP_NAME']

#To print the error of query conditon, only print once
for err_id in range(len(queryCondition['GENERAL_ERROR'])):
    print queryCondition['GENERAL_ERROR'][err_id]

In [ ]:
#To get data and draw graph by data.
if(ISOK_ALL_PARA and (REPORT_TYPE == 'ALL' or REPORT_TYPE == 'TBSP_TABLE')):
    DB_CONN_ID = DB_SET[0] 
    #To get data from the target database to assemble a data frame that will be used for the following graph
    df_interval = %sql with temp as( select date(IBMIOCM_TIMESTAMP) as date, \
                       hour(IBMIOCM_TIMESTAMP) as hour, \
                       tbsp_name, \
                       (decimal(tbsp_total_pages,17,2) * decimal(tbsp_page_size,17,2)) / 1024 /1024 as TBSP_TOTAL_SIZE_MB, \
                       (decimal(tbsp_used_pages,17,2) * decimal(tbsp_page_size,17,2)) / 1024 / 1024 as TBSP_USED_SIZE_MB, \
                       (decimal(tbsp_free_pages,17,2) * decimal(tbsp_page_size,17,2)) / 1024 / 1024 as TBSP_FREE_SIZE_MB, \
                       (decimal(tbsp_usable_pages,17,2) * decimal(tbsp_page_size,17,2)) / 1024 /1024 as TBSP_USABLE_SIZE_MB, \
                       case when tbsp_usable_pages > 0 then decimal(round((decimal(tbsp_used_pages,17,2) / decimal(tbsp_usable_pages,17,2)),2),17,2) * 100 \
                       when tbsp_usable_pages = 0 then 0 else NULL end as TBSP_UTILIZATION_PERCENT \
                       from IBMIOCM.DB2LUW_MONTABLESPACE_HIS \
                       where IBMIOCM_DATABASE = '{DB_CONN_ID}' and IBMIOCM_TIMESTAMP >= '{START_TIME}' and  IBMIOCM_TIMESTAMP < '{END_TIME}' \
                       order by date(IBMIOCM_TIMESTAMP), hour(IBMIOCM_TIMESTAMP)), temp_stg \
                       as ( select row_number()over(partition by date, hour \
                       order by TBSP_USED_SIZE_MB desc) as \
                       row_id, date, hour, tbsp_name, TBSP_TOTAL_SIZE_MB, TBSP_USED_SIZE_MB, TBSP_FREE_SIZE_MB, TBSP_USABLE_SIZE_MB, TBSP_UTILIZATION_PERCENT \
                       from temp) \
                       select date, hour, tbsp_name, TBSP_TOTAL_SIZE_MB, TBSP_USED_SIZE_MB, TBSP_FREE_SIZE_MB, TBSP_USABLE_SIZE_MB, TBSP_UTILIZATION_PERCENT, row_id as rank \
                       from temp_stg where row_id <= 3; 
            
    if os.path.exists("tbsp.csv"):
        !rm tbsp.csv
    if df_interval.empty:
        print 'For Tablespace: The query result is empty, please check your query parameters.\n'
    else:
        #To judge whether file exists or not and if yes, delete it, if not save the data into a file storage.csv
        df_interval.to_csv("tbsp.csv", index_label = "INDEX")

In [ ]:
#To get data and draw graph by data.
if(ISOK_ALL_PARA and (REPORT_TYPE == 'ALL' or REPORT_TYPE == 'TABLESPACE')):
    DB_CONN_ID = DB_SET[0]
    #dataframe_week = %sql select year(collected) as year, week(collected) as week, substr(dbconn_id,1,30) as conn_name, date(min(collected)) as begin_date, date(max(collected)) as end_date, sum(total_cpu_usec_delta) / 1000000.0 as cpu_sec, sum(logical_reads_delta) as logical_reads, sum(physical_reads_delta) as physical_reads, sum(act_completed_total_delta) as activities, sum(total_app_commits_delta) as commits, sum(total_act_time_delta) / (sum(act_aborted_total_delta) + sum(act_completed_total_delta)) as avg_activity_time_msec from ibm_dsm_views.throughput_all group by year(collected), week(collected), dbconn_id order by year(collected), week(collected), dbconn_id 
    df_interval = %sql select date(IBMIOCM_TIMESTAMP) as date, \
                       hour(IBMIOCM_TIMESTAMP) as hour, \
                       tbsp_name, (tbsp_total_pages * tbsp_page_size) / 1024 /1024 as TBSP_TOTAL_SIZE_MB, \
                       (tbsp_used_pages * tbsp_page_size) / 1024 / 1024 as TBSP_USED_SIZE_MB, \
                       (tbsp_free_pages * tbsp_page_size) / 1024 / 1024 as TBSP_FREE_SIZE_MB, \
                       (tbsp_usable_pages * tbsp_page_size) / 1024 /1024 as TBSP_USABLE_SIZE_MB, \
                       case when tbsp_usable_pages > 0 then \
                       decimal(round((decimal(tbsp_used_pages,17,2) / decimal(tbsp_usable_pages,17,2)),2),17,2) * 100 \
                       when tbsp_usable_pages = 0 then 0 else NULL end as TBSP_UTILIZATION_PERCENT \
                       from IBMIOCM.DB2LUW_MONTABLESPACE_HIS \
                       where IBMIOCM_DATABASE='{DB_CONN_ID}' and tbsp_name = '{TBSP_NAME}' and IBMIOCM_TIMESTAMP >= '{START_TIME}' and IBMIOCM_TIMESTAMP < '{END_TIME}' \
                       order by date(IBMIOCM_TIMESTAMP), hour(IBMIOCM_TIMESTAMP)
    if os.path.exists("tablespace.csv"):
        !rm tablespace.csv
    if df_interval.empty:
        print 'For Tablespace Table: The query result is empty, please check your query parameters.\n'
    else:
        #To save tablespace data for generating table
        df_interval.to_csv("tablespace.csv", index_label = "INDEX")
        dataframe_tbsp = %sql select date(IBMIOCM_TIMESTAMP) as date, \
                              hour(IBMIOCM_TIMESTAMP) as hour, \
                              tbsp_name, \
                              (tbsp_total_pages * tbsp_page_size) / 1024 /1024 as TBSP_TOTAL_SIZE_MB, \
                              (tbsp_used_pages * tbsp_page_size) / 1024 / 1024 as TBSP_USED_SIZE_MB, \
                              (tbsp_free_pages * tbsp_page_size) / 1024 / 1024 as TBSP_FREE_SIZE_MB, \
                              (tbsp_usable_pages * tbsp_page_size) / 1024 /1024 as TBSP_USABLE_SIZE_MB, \
                              case when tbsp_usable_pages > 0 then decimal(round((decimal(tbsp_used_pages,17,2) / decimal(tbsp_usable_pages,17,2)),2),17,2) * 100 \
                              when tbsp_usable_pages = 0 then 0 else NULL end as TBSP_UTILIZATION_PERCENT \
                              from IBMIOCM.DB2LUW_MONTABLESPACE_HIS \
                              where IBMIOCM_DATABASE='{DB_CONN_ID}' and tbsp_name = '{TBSP_NAME}' and IBMIOCM_TIMESTAMP < '{END_TIME}'\
                              order by date(IBMIOCM_TIMESTAMP), hour(IBMIOCM_TIMESTAMP)
        
        x_hour = dataframe_tbsp['HOUR'].values
        x_hour_list = list(x_hour)
        y_tbsp_util_percent = dataframe_tbsp['TBSP_UTILIZATION_PERCENT'].values
        y_tbsp_util_percent_list = list(y_tbsp_util_percent)
        date_all = list(dataframe_tbsp['DATE'].values)
        
        ori_datetime_str = commonUtil.get_original_datatime_str(date_all, x_hour_list)
        #To set predict_count according to INTERVAL and max value is 10
        if(INTERVAL <= 10):
            predict_count = float('%.1f' % INTERVAL)
        else:
            predict_count = 10.0

        #To form datetime_str from date_ori for indexing
        datetime_str = []
        for id_1 in range(len(date_all)):
            tmp_hour = str(x_hour_list[id_1])
            if(len(tmp_hour) == 1):
                tmp_hour = '0' + tmp_hour
            tmp_dt = str(date_all[id_1]) + ' ' + tmp_hour
            datetime_str.append(tmp_dt)
            
        '''
        To complement data for no data some hours
        '''
        #To get start timestamp First 
        min_dt_str = datetime_str[0] + ':00:00'
        min_tuple = time.strptime(min_dt_str, '%Y-%m-%d %H:00:00')
        min_timestamp = time.mktime(min_tuple)
            
        #To get end timestamp 
        max_tuple = time.strptime(END_TIME[0:13] + ":00:00", '%Y-%m-%d %H:00:00')
        max_timestamp = time.mktime(max_tuple)

        hour_diff = int((max_timestamp - min_timestamp) / 3600)
        #clear date_all
        date_all = []
        #clear x_hour_list
        x_hour_list = []
        ori_dt_str = []
        all_tbsp_util_list_1 = []

        #Reassign 4 variables above
        for id_2 in range(hour_diff):
            tmp_st = min_timestamp + id_2 * 3600.0
            #To change timestamp to datetime string
            tmp_datetime = datetime.datetime.fromtimestamp(tmp_st)
            tmp_datetime_str = tmp_datetime.strftime("%Y-%m-%d %H:00:00")
            tmp_date_str = tmp_datetime_str[0:10]
            tmp_hour_str = tmp_datetime_str[11:13]
            ori_dt_str.append(tmp_date_str + ' ' + tmp_hour_str)
            date_all.append(tmp_date_str)
            x_hour_list.append(tmp_hour_str)

            '''
            Below code is for handling y-axis's data
            If there is no data at this hour,0.0 will be filled into
            '''
            if tmp_datetime_str in ori_datetime_str:
                tmp_index = ori_datetime_str[tmp_datetime_str]
                all_tbsp_util_list_1.append(y_tbsp_util_percent_list[tmp_index])
            else:
                all_tbsp_util_list_1.append(1.0)
            
        df_final = pd.Series(all_tbsp_util_list_1)
        df_final.index = pd.PeriodIndex(start = ori_dt_str[0], end = ori_dt_str[len(ori_dt_str) - 1])
        tbsp_model = sm.tsa.ARIMA(df_final,(7,0,1)).fit()
            
        #To get end_timestamp from ori_dt_str for preparing to form x_ticks and x_ticks_lables
        end_dt_str = ori_dt_str[len(ori_dt_str) - 1] + ":00:00"
        end_tuple = time.strptime(end_dt_str, '%Y-%m-%d %H:00:00')
        end_timestamp = time.mktime(end_tuple)
            
        #The count of output original data 
        ori_hour = None
        ori_hour_st = end_timestamp - 3600.0 * (INTERVAL - 1)
        tmp_datetime = datetime.datetime.fromtimestamp(ori_hour_st)
        ori_hour = tmp_datetime.strftime("%Y-%m-%d %H:00:00")[0:13]

        #The count of predict data 
        predict_hour = None
        predict_hour_st = None
        #If INTERVAL>10，we will predict only 10 hours
        #If not we we will predict INTERVAL hours
        if(INTERVAL <= 10):
            predict_hour_st = end_timestamp + 3600.0 * INTERVAL
        else:
            predict_hour_st = end_timestamp + 3600 * predict_count
        tmp_datetime = datetime.datetime.fromtimestamp(predict_hour_st)
        predict_hour = tmp_datetime.strftime("%Y-%m-%d %H:00:00")[0:13]
            
        #To get predict data
        predict_array = tbsp_model.predict(ori_dt_str[len(ori_dt_str) - 1], predict_hour, dynamic=True)
        predict_array_list = list(predict_array)
        predict_array_list[0] = all_tbsp_util_list_1[len(predict_array_list) - 1]
        predict_array = np.asarray(predict_array_list[1:len(predict_array_list)])
        all_tbsp_util_list_1 = all_tbsp_util_list_1[-INTERVAL:] + list(predict_array)
           
        #To set x ticks and x label
        loop_count = int((predict_hour_st - ori_hour_st)/3600.0)
        date_all = []
        x_hour = []
        for id_3 in range(loop_count + 1):
            tmp_hour_st = ori_hour_st + id_3 * 3600.0
            tmp_datetime = datetime.datetime.fromtimestamp(tmp_hour_st)
            date_all.append(tmp_datetime.strftime("%Y-%m-%d %H:00:00")[0:10])
            x_hour.append(tmp_datetime.strftime("%Y-%m-%d %H:00:00")[11:13])

        x_ticks = []
        x_ticks_lables = []
        if len(x_ticks) == 0:
            commonUtil.format_x_axis(date_all, x_hour, x_ticks, x_ticks_lables)

        #To declare a Sketchpad
        fig = pl.figure()
        #To declare an ax container as a drawing paper
        ax_tbsp_util= fig.add_subplot(111)

        data_size = len(x_ticks)
        if(data_size <= 20):
            fig.set_size_inches(12,6)
        elif(data_size <= 40):
            fig.set_size_inches(16,6)
        elif(data_size <= 60):
            fig.set_size_inches(18,7)
        elif(data_size <=100):
            fig.set_size_inches(22,7)

        #To set the title/label/grid for the graph
        figure_title = 'The Table Space Utilization Percent by Hour\n'
        pl.title(figure_title, fontsize = 14, fontweight = 'bold')
        x_lable = 'Hours'
        #To set x-axis label
        pl.xlabel(x_lable)
        #To set y-axis label
        pl.ylabel(u'TBSP_UTILIZATION_PERCENT %')
        #To set grid line style according to your requirement
        pl.grid(True, ls = '--', color = '#2c628b', alpha = 0.05)
        pl.xticks(x_ticks, x_ticks_lables, rotation = 90)
        
        #To scatter the data of y-axis and mark the data point
        for id_5 in range(len(all_tbsp_util_list_1)):
                if(all_tbsp_util_list_1[id_5] == 0.0):#If no data,drawing a empty circle
                    if id_5 >= len(all_tbsp_util_list_1) - predict_count and id_5 < len(all_tbsp_util_list_1):
                        #This is the data of prediction and empty
                        pl.scatter(x_ticks[id_5], all_tbsp_util_list_1[id_5], c = 'r') 
                    else:
                        #This is the real data of y_max_log_percent and empty
                        pl.scatter(x_ticks[id_5], all_tbsp_util_list_1[id_5], c = '', marker = 'o', edgecolors = 'r', s = 50)
                else:#Data not empty
                    if id_5 >= len(all_tbsp_util_list_1) - predict_count and id_5 < len(all_tbsp_util_list_1):
                        #This is the data of prediction and not empty
                        pl.scatter(x_ticks[id_5], all_tbsp_util_list_1[id_5], c = 'r') 
                        pl.text(x_ticks[id_5], all_tbsp_util_list_1[id_5], '%.0f' % all_tbsp_util_list_1[id_5], fontsize = 9)
                    else:
                        #This is the real data of y_max_log_percent and not empty
                        pl.scatter(x_ticks[id_5], all_tbsp_util_list_1[id_5], c = '#4c78fb') 
                        pl.text(x_ticks[id_5], all_tbsp_util_list_1[id_5], '%.0f' % all_tbsp_util_list_1[id_5], fontsize = 9)

        #Below two variable is used for storeing magnified data
        xnew_hour = []
        ynew_tbsp_util_percent = []
        #Expand each x axis data 20 times
        xnew_hour = np.linspace(np.asarray(x_ticks).min(), np.asarray(x_ticks).max(), np.asarray(x_ticks).size*20) 
        #Handle the data of new y axis data
        ynew_tbsp_util_percent = spline(np.asarray(x_ticks), np.asarray(all_tbsp_util_list_1), xnew_hour)
        ynew_tbsp_util_percent_list = list(ynew_tbsp_util_percent)
        #No negative value for y-axis
        for y_idx in range(len(ynew_tbsp_util_percent_list)):
            if (ynew_tbsp_util_percent_list[y_idx] < 0.0):
                ynew_tbsp_util_percent_list[y_idx] = 0.0
        ynew_tbsp_util_percent = np.asarray(ynew_tbsp_util_percent_list)

        #Fill the gragh according to your requirement
        tbsp_ratio = len(list(predict_array)) / float('%.1f' % len(all_tbsp_util_list_1))
        #pl.fill_between(xnew_hour, ynew_tbsp_util_percent, where=(xnew_hour.min() < xnew_hour) & (xnew_hour < xnew_hour.max() * (1.0 - tbsp_ratio)), color = '#4c78fb', alpha = 0.15)
        pl.fill_between(xnew_hour, ynew_tbsp_util_percent, where=(xnew_hour.max() * (1.0 - tbsp_ratio) < xnew_hour) & (xnew_hour < xnew_hour.max()), color = '#2c628b', alpha = 0.1)
        #Draw curve graph
        pl.plot(xnew_hour, ynew_tbsp_util_percent, color = '#2c628b')
        pl.yticks(np.arange(0, 101, 10))
        pl.show()

In [ ]:
#To get data and draw graph by data.
if(ISOK_ALL_PARA and (REPORT_TYPE == 'ALL' or REPORT_TYPE == 'DATABASE')):
    #To store all table spaces' usage information for drawing graph
    all_db_util_list = []
    #x ticks sorted by asc
    x_ticks = None
    #real x ticks label
    x_ticks_labels = None
    #per legend related with each table space
    legend_title_db = []
    #To get data from the target table to assemble a data frame that will be used for table view
    #df_interval = %sql with temp as (select ibmiocm_database as dbname, IBMIOCM_TIMESTAMP, decimal(round(decimal((0.000000954*SUM(TBSP_USED_PAGES * TBSP_PAGE_SIZE)),17,2),2), 17,2) AS USAGE_GB FROM IBMIOCM.DATABASE_HIS where IBMIOCM_TIMESTAMP >= '{START_TIME}' and  IBMIOCM_TIMESTAMP < '{END_TIME}'  group by ibmiocm_database, IBMIOCM_TIMESTAMP ORDER BY ibmiocm_database, IBMIOCM_TIMESTAMP), temp_stg as(select dbname as dbname, date(IBMIOCM_TIMESTAMP) as date, hour(IBMIOCM_TIMESTAMP) as hour, max(USAGE_GB) as MAX_USAGE_GB from temp group by dbname, date(IBMIOCM_TIMESTAMP), hour(IBMIOCM_TIMESTAMP) ORDER BY dbname, date, hour), temp_stg2 as(select row_number() over (partition by date order by MAX_USAGE_GB desc) as row_id, dbname, date, hour, MAX_USAGE_GB from temp_stg) select dbname, date, hour, MAX_USAGE_GB, row_id as rank from temp_stg2 where row_id <= 5
    df_interval = %sql with temp as (select ibmiocm_database as dbname, \
                       IBMIOCM_TIMESTAMP, \
                       decimal(round(decimal((0.000000954*SUM(TBSP_USED_PAGES * TBSP_PAGE_SIZE)),17,2),2), 17,2) AS USAGE_GB \
                       FROM IBMIOCM.DATABASE_HIS \
                       where IBMIOCM_TIMESTAMP >= '{START_TIME}' and  IBMIOCM_TIMESTAMP < '{END_TIME}'  \
                       group by ibmiocm_database, IBMIOCM_TIMESTAMP \
                       ORDER BY ibmiocm_database, IBMIOCM_TIMESTAMP), \
                       temp_stg as (\
                       select dbname as dbname, \
                       date(IBMIOCM_TIMESTAMP) as date, \
                       hour(IBMIOCM_TIMESTAMP) as hour, \
                       max(USAGE_GB) as MAX_USAGE_GB \
                       from temp \
                       group by dbname, date(IBMIOCM_TIMESTAMP), hour(IBMIOCM_TIMESTAMP) \
                       ORDER BY dbname, date, hour), temp_stg2 as(select row_number() over \
                       (partition by date,hour order by MAX_USAGE_GB desc) as row_id, \
                       dbname, date, hour, MAX_USAGE_GB from temp_stg) \
                       select dbname, date, hour, MAX_USAGE_GB, row_id as rank \
                       from temp_stg2 \
                       where row_id <= 5
                
    if os.path.exists("db.csv"):
        !rm db.csv
    if df_interval.empty:
        print 'For databases: The query result is empty, please check your query parameters.\n'
    else:
        #To get top5 database
        db_name_list = list(df_interval['DBNAME'].values)
        db_name_set = set(db_name_list)
        db_name_list = list(db_name_set)
        #To judge whether file exists or not and if yes, delete it, if not save the data into a file db.csv
        df_interval.to_csv("db.csv", index_label = "INDEX")
        for id_0 in range(len(db_name_list)):
            tmp_db_name_str = str(db_name_list[id_0])
            legend_title_db.append(tmp_db_name_str)
            df_db_hour = %sql with temp as (select ibmiocm_database as dbname, IBMIOCM_TIMESTAMP, decimal(round(decimal((0.000000954*SUM(TBSP_USED_PAGES * TBSP_PAGE_SIZE)),17,2),2), 17,2) AS USAGE_GB FROM IBMIOCM.DATABASE_HIS where ibmiocm_database = '{tmp_db_name_str}' and IBMIOCM_TIMESTAMP < '{END_TIME}' group by ibmiocm_database, IBMIOCM_TIMESTAMP ORDER BY ibmiocm_database, IBMIOCM_TIMESTAMP), temp_stg as(select dbname as dbname, date(IBMIOCM_TIMESTAMP) as date, hour(IBMIOCM_TIMESTAMP) as hour, max(USAGE_GB) as MAX_USAGE_GB from temp group by dbname, date(IBMIOCM_TIMESTAMP), hour(IBMIOCM_TIMESTAMP) ORDER BY dbname, date, hour), temp_stg2 as(select row_number() over (partition by date order by MAX_USAGE_GB desc) as row_id, dbname, date, hour, MAX_USAGE_GB from temp_stg) select dbname, date, hour, MAX_USAGE_GB, row_id as rank from temp_stg2
            #To get the hour data as x-axis from the data frame and convert the data into a numpy array
            x_hour = df_db_hour['HOUR'].values
            x_hour_list = list(x_hour)
            #To get the data as y-axis from the data frame and convert the data into a numpy array
            y_db_util_percent = df_db_hour['MAX_USAGE_GB'].values
            #To chenage y_tbsp_util_percent into y_db_util_percent_list
            y_db_util_percent_list = list(y_db_util_percent)
            
            date_all = list(df_db_hour['DATE'].values)
            
            ori_datetime_str = commonUtil.get_original_datatime_str(date_all, x_hour_list)
            
            #To set predict_count according to INTERVAL and max value is 10
            if(INTERVAL <= 10):
                predict_count = float('%.1f' % INTERVAL)
            else:
                predict_count = 10.0
            
            #To form datetime_str from date_ori for indexing
            datetime_str = []
            for id_1 in range(len(date_all)):
                tmp_hour = str(x_hour_list[id_1])
                if(len(tmp_hour) == 1):
                    tmp_hour = '0' + tmp_hour
                tmp_dt = str(date_all[id_1]) + ' ' + tmp_hour
                datetime_str.append(tmp_dt)
            
            '''
            To complement data for no data some hours
            '''
            #To get start timestamp First 
            min_dt_str = datetime_str[0] + ':00:00'
            min_tuple = time.strptime(min_dt_str, '%Y-%m-%d %H:00:00')
            min_timestamp = time.mktime(min_tuple)
            
            #To get end timestamp 
            max_tuple = time.strptime(END_TIME[0:13] + ":00:00", '%Y-%m-%d %H:00:00')
            max_timestamp = time.mktime(max_tuple)
           
            hour_diff = int((max_timestamp - min_timestamp) / 3600)
            all_db_util_list_1 = []
            #clear date_all
            date_all = []
            #clear x_hour_list
            x_hour_list = []
            ori_dt_str = []
            #Reassign three variables above
            for id_2 in range(hour_diff):
                tmp_st = min_timestamp + id_2 * 3600.0
                #To change timestamp to datetime string
                tmp_datetime = datetime.datetime.fromtimestamp(tmp_st)
                tmp_datetime_str = tmp_datetime.strftime("%Y-%m-%d %H:00:00")
                tmp_date_str = tmp_datetime_str[0:10]
                tmp_hour_str = tmp_datetime_str[11:13]
                ori_dt_str.append(tmp_date_str + ' ' + tmp_hour_str)
                date_all.append(tmp_date_str)
                x_hour_list.append(tmp_hour_str)

                '''
                Below code is for handling y-axis's data
                If there is no data at this hour,0.0 will be filled into
                '''
                if tmp_datetime_str in ori_datetime_str:
                    tmp_index = ori_datetime_str[tmp_datetime_str]
                    all_db_util_list_1.append(y_db_util_percent_list[tmp_index])
                else:
                    all_db_util_list_1.append(1.0)
            #all_db_util_list.append(all_db_util_list_1)       
            df_final = pd.Series(all_db_util_list_1)
            df_final.index = pd.PeriodIndex(start = ori_dt_str[0], end = ori_dt_str[len(ori_dt_str) - 1])
            db_model = sm.tsa.ARIMA(df_final, (7,0,1)).fit()
            
            #To get end_timestamp from ori_dt_str for preparing to form x_ticks and x_ticks_lables
            end_dt_str = ori_dt_str[len(ori_dt_str) - 1] + ":00:00"
            end_tuple = time.strptime(end_dt_str, '%Y-%m-%d %H:00:00')
            end_timestamp = time.mktime(end_tuple)
            
            #The count of output original data 
            ori_hour = None
            ori_hour_st = end_timestamp - 3600.0 * (INTERVAL - 1)
            tmp_datetime = datetime.datetime.fromtimestamp(ori_hour_st)
            ori_hour = tmp_datetime.strftime("%Y-%m-%d %H:00:00")[0:13]

            #The count of predict data 
            predict_hour = None
            predict_hour_st = None
            #If INTERVAL>10，we will predict only 10 hours
            #If not we we will predict INTERVAL hours
            if(INTERVAL <= 10):
                predict_hour_st = end_timestamp + 3600.0 * INTERVAL
            else:
                predict_hour_st = end_timestamp + 3600 * predict_count
            tmp_datetime = datetime.datetime.fromtimestamp(predict_hour_st)
            predict_hour = tmp_datetime.strftime("%Y-%m-%d %H:00:00")[0:13]
            
            #To get predict data
            predict_array = db_model.predict(ori_dt_str[len(ori_dt_str) - 1], predict_hour, dynamic=True)
            predict_array_list = list(predict_array)
            #If predict_array including NaN value,change them into 0.0
            for i in range(len(predict_array_list)):
                if np.isnan(float(predict_array_list[i])):
                    predict_array_list[i] = 0.0
            #predict_array = np.asarray(predict_array_list[1:len(predict_array_list)])
            all_db_util_list_1 = all_db_util_list_1[-INTERVAL:] + predict_array_list[1:len(predict_array_list)]
            all_db_util_list.append(all_db_util_list_1)
            
            #To set x ticks and x label
            loop_count = int((predict_hour_st - ori_hour_st)/3600.0)
            date_all = []
            x_hour = []
            for id_3 in range(loop_count + 1):
                tmp_hour_st = ori_hour_st + id_3 * 3600.0
                tmp_datetime = datetime.datetime.fromtimestamp(tmp_hour_st)
                date_all.append(tmp_datetime.strftime("%Y-%m-%d %H:00:00")[0:10])
                x_hour.append(tmp_datetime.strftime("%Y-%m-%d %H:00:00")[11:13])
            x_ticks = []
            x_ticks_lables = []
            #if len(x_ticks) == 0:
            commonUtil.format_x_axis(date_all, x_hour, x_ticks, x_ticks_lables)
        
        if len(all_db_util_list) > 0:
            #To declare color set
            color_set = ['#ffb90f', '#a6266e', '#4c78fb', '#2c628b', '#60bdae']
            #To declare a Sketchpad
            fig = pl.figure()
            #To declare an ax container as a drawing paper
            ax_db_util= fig.add_subplot(111)

            data_size = len(x_ticks)
            if(data_size <= 20):
                fig.set_size_inches(12,6)
            elif(data_size <= 40):
                fig.set_size_inches(16,6)
            elif(data_size <= 60):
                fig.set_size_inches(18,7)
            elif(data_size <=100):
                fig.set_size_inches(22,7)

            #To set the title/label/grid for the graph
            figure_title = 'The Max Database Usage by Hour\n'
            pl.title(figure_title, fontsize = 14, fontweight = 'bold')
            x_lable = 'Hours'
            #To set x-axis label
            pl.xlabel(x_lable)
            #To set y-axis label
            pl.ylabel(u'DATABASE MAX_USAGE_GB')
            #To set grid line style according to your requirement
            pl.grid(True, ls = '--', color = '#2c628b', alpha = 0.05)
            pl.xticks(x_ticks, x_ticks_lables, rotation = 90)
        
            #Below two variable is used for storeing magnified data
            xnew_hour = []
            ynew_db_util_percent = []
        
            for id_4 in range(len(all_db_util_list)):
                #To scatter the data of y-axis and mark the data point
                for id_5 in range(len(all_db_util_list[id_4])):
                    if(all_db_util_list[id_4][id_5] == 0.0):#If no data,drawing a empty circle
                        if id_5 >= len(all_db_util_list[id_4]) - predict_count and id_5 < len(all_db_util_list[id_4]):
                            #This is the data of prediction and empty
                            pl.scatter(x_ticks[id_5], all_db_util_list[id_4][id_5], c = 'r') 
                        else:
                            #This is the real data of y_max_log_percent and empty
                            pl.scatter(x_ticks[id_5], all_db_util_list[id_4][id_5], c = '', marker = 'o', edgecolors = 'r', s = 50)
                    else:#Data not empty
                        if id_5 >= len(all_db_util_list[id_4]) - predict_count and id_5 < len(all_db_util_list[id_4]):
                            #This is the data of prediction and not empty
                            pl.scatter(x_ticks[id_5], all_db_util_list[id_4][id_5], c = 'r') 
                            pl.text(x_ticks[id_5], all_db_util_list[id_4][id_5], '%.0f' % all_db_util_list[id_4][id_5], fontsize = 9)
                        else:
                            #This is the real data of y_max_log_percent and not empty
                            pl.scatter(x_ticks[id_5], all_db_util_list[id_4][id_5], c = '#4c78fb') 
                            pl.text(x_ticks[id_5], all_db_util_list[id_4][id_5], '%.0f' % all_db_util_list[id_4][id_5], fontsize = 9)

                #Expand each x axis data 20 times
                xnew_hour = np.linspace(np.asarray(x_ticks).min(), np.asarray(x_ticks).max(), np.asarray(x_ticks).size*20) 
                #Handle the data of new y axis data
                ynew_db_util_percent = spline(np.asarray(x_ticks), np.asarray(all_db_util_list[id_4]), xnew_hour)
                ynew_db_util_percent_list = list(ynew_db_util_percent)
                #No negative value for y-axis
                for y_idx in range(len(ynew_db_util_percent_list)):
                    if (ynew_db_util_percent_list[y_idx] < 0.0):
                        ynew_db_util_percent_list[y_idx] = 0.0
                ynew_db_util_percent = np.asarray(ynew_db_util_percent_list)
                #Fill the gragh according to your requirement
                db_ratio = len(list(predict_array)) / float('%.1f' % len(all_db_util_list[id_4]))
                #pl.fill_between(xnew_hour, ynew_db_util_percent, where=(xnew_hour.min() < xnew_hour) & (xnew_hour < xnew_hour.max() * (1.0 - tbsp_ratio)), color = '#4c78fb', alpha = 0.15)
                pl.fill_between(xnew_hour, ynew_db_util_percent, where=(xnew_hour.max() * (1.0 - db_ratio) < xnew_hour) & (xnew_hour < xnew_hour.max()), color = '#2c628b', alpha = 0.1)
                #Draw curve graph
                pl.plot(xnew_hour, ynew_db_util_percent, color = color_set[id_4])
            #Set the legends for the both graphs
            box_db = ax_db_util.get_position()
            ax_db_util.set_position([box_db.x0, box_db.y0 + box_db.height * 0.1, box_db.width, box_db.height * 0.9])
            ax_db_util.legend(legend_title_db, fontsize = 11, loc = 'upper center', bbox_to_anchor=(0.5,1.08), fancybox = True, shadow = True, ncol = len(legend_title_db))
            #pl.yticks(np.arange(0, 101, 10))
            pl.show()